In [3]:
import pandas as pd
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
import os
import matplotlib.pyplot as plt # Para graficar
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import joblib # Para guardar el modelo
import numpy as np

print("Librerías importadas exitosamente.")

Librerías importadas exitosamente.
